https://arxiv.org/pdf/1703.01365.pdf

## Imports

In [3]:
import argparse
import copy
import csv
import importlib
import itertools
import operator
import os
import random
import time
from math import e, exp, log, pi, sqrt
from os.path import *

import keras.layers as layers
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import scipy
import sklearn.decomposition
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
from numpy import diag, matmul
from scipy.stats import ttest_ind
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import cnn_analyzer as cnna
import cnn_builder as cbuild
import cnn_runner as crun
import config
import dr_methods as drm
import inference_methods_squash as im
import niftiutils.helper_fxns as hf
import niftiutils.private as prv

%matplotlib inline

In [4]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(cnna)
C = config.Config()

In [20]:
target_dir = "D:\\feature_analysis"

importlib.reload(cbuild)
model = keras.models.load_model(join(C.model_dir, "model_reader1.hdf5")) #models_305

#model_conv1 = cbuild.build_pretrain_model(model, padding = ['same','same'], last_layer=-5, add_activ=True)
#model_conv2 = cbuild.build_pretrain_model(model, padding = ['same','same'], last_layer=-4, add_activ=True)
#model_conv3 = cbuild.build_pretrain_model(model, padding = ['same','same'], last_layer=-3, add_activ=True)
model_dense = cbuild.pretrain_cnn(model, padding = ['same','valid'], last_layer=-2, add_activ=True)

In [5]:
orig_data_dict, num_samples = cbuild._collect_unaug_data()

features_by_cls, feat_count = cnna.collect_features()
#feat_count.pop("central scar")
feat_count.pop("homogeneous texture")
all_features = list(feat_count.keys())
cls_features = {f: [c for c in C.classes_to_include if f in features_by_cls[c]] for f in all_features}

num_annotations = 10
Z_features = cnna.get_annotated_files(features_by_cls, num_annotations)
#Z_features.pop("central scar")
Z_features.pop("homogeneous texture")

array(['E105368141_0', '12362250_6', 'E100215900_4', 'E106182827_0',
       '12362250_2', '12362250_4', 'E105918926_0', 'E100215900_6',
       'E104490005_0', '12362250_0'], dtype='<U12')

In [ ]:
sorted(all_features)

In [10]:
import voi_methods as vm
for f in Z_features:
    save_path = "D:\\labeled_features\\"+f
    if not exists(save_path):
        os.makedirs(save_path)
    vm.save_vois_as_imgs(lesion_ids=Z_features[f], save_dir=save_path)

In [ ]:
feature_mapping = {'arterial phase enhancement',
 'central scar': 'central scar',
 'enhancing rim/capsule',
 'heterogeneous texture',
 'hyperintense mass on delayed phase',
 'hypointense mass without enhancement',
 'infiltrative growth',
 'isointense on venous phase',
 'nodular growth',
 'nodular or discontinuous enhancement',
 'progressive centripetal filling',
 'progressive uniform enhancement',
 'sharp margins',
 'spherical hypointense mass',
 'washout'}

In [5]:
# best model
#Z_test = ['E106097391_0', 'E104978772_1', '12900535_0', 'E100150242_0', 'E105490014_0', 'E103147618_0', 'E103510187_0', 'E104657225_0', 'E100551966_0', 'E101388602_0', 'E100215900_8', 'E100215900_7', 'E104045692_0', '13104521_0', 'E100383453_0', '12943286_0', '12271995_0', 'E102315724_0', 'E104949189_0', 'E100511083_1', 'E101579471_0', '13018986_1', '13203550_8', '13112385_0', '12712463_0', '12361082_0', '13028374_0', 'E103985934_1', 'E100529980_0', '12042703_3', '12961059_0', 'E105724706_2', 'E100592424_2', 'E103104254_0', 'E104546069_0', 'E101665217_1', '12090000_0', 'E100592424_1', '12961059_1', 'E105474285_0', '12502068_1', 'E100814791_0', 'E102613189_0', 'E105427046_0', 'E102881031_1', 'E102929168_0', 'E102310482_0', 'E102095465_0', 'E101811299_0', 'E104737273_0', '12890053_0', 'E100168661_1', '12637865_0', 'E100168661_2', '12239783_0', '12707781_0', '12706568_1', '12823036_0', '12404081_0', '12365693_1']
# reader
Z_test = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']

num_features = len(all_features) # number of features

all_imgs = [orig_data_dict[cls][0] for cls in C.classes_to_include]
all_imgs = np.array(hf.flatten(all_imgs))

all_lesionids = [orig_data_dict[cls][1] for cls in C.classes_to_include]
all_lesionids = np.array(hf.flatten(all_lesionids))

test_indices = np.where(np.isin(all_lesionids, Z_test))[0]

fixed_indices = np.empty([num_features, num_annotations])
for f_ix,f in enumerate(all_features):
    if len(np.where(np.isin(all_lesionids, random.sample(set(Z_features[f]), num_annotations)))[0]) < 10:
        print(f,Z_features[f])
    fixed_indices[f_ix, :] = np.where(np.isin(all_lesionids, random.sample(set(Z_features[f]), num_annotations)))[0]
fixed_indices = fixed_indices.astype(int)

x_test = all_imgs[test_indices]
z_test = all_lesionids[test_indices]
len(z_test)

60

In [6]:
voi_df = drm.get_voi_dfs()[0]
Z = np.concatenate([orig_data_dict[cls][1] for cls in C.classes_to_include], 0)
aug_factor = 20
num_samples = aug_factor*len(Z)

all_dense = np.empty([num_samples,100])
all_conv3_sh = np.empty([num_samples,128*4])
all_conv2_sh = np.empty([num_samples,128*4])
all_conv3_ch = np.empty([num_samples,128])
all_conv2_ch = np.empty([num_samples,128])
all_conv1_sh = np.empty([num_samples,64*12])
all_conv1_ch = np.empty([num_samples,64*3])

for img_id in range(len(Z)):
    voi_row = voi_df.loc[Z[img_id]]
    for aug_id in range(aug_factor):
        img = np.load(os.path.join(C.aug_dir, voi_row['cls'], "%s_%d.npy" % (Z[img_id], aug_id)))
        img = np.expand_dims(img, 0)
        ix = img_id*aug_factor + aug_id
        
        activ = model_dense.predict(img)[0]
        #activ[activ < 0] = 0
        all_dense[ix] = activ
        
        #activ = model_conv3.predict(img)[0]
        #activ[activ < 0] = 0
        #all_conv3_ch[ix] = activ.mean((0,1,2))
        #all_conv3_sh[ix] = cnna.get_shells(activ, D)
        
        #activ = model_conv2.predict(img)[0]
        #activ[activ < 0] = 0
        #all_conv2_ch[ix] = activ.mean((0,1,2))
        #all_conv2_sh[ix] = cnna.get_shells(activ, D)
        
        #activ = model_conv1.predict(img)[0]
        #activ[activ < 0] = 0
        #all_conv1_ch[ix] = activ.mean((0,1,2))
        #all_conv1_sh[ix] = cnna.get_shells(activ, D)

In [7]:
feature_dense = {f:np.empty([0,100]) for f in all_features}
feature_conv3_ch = {f:np.empty([0,128]) for f in all_features}
feature_conv3_sh = {f:np.empty([0,128*4]) for f in all_features}
feature_conv2_ch = {f:np.empty([0,128]) for f in all_features}
feature_conv1_ch = {f:np.empty([0,64*3]) for f in all_features}

aug_factor = 100
for f in all_features:
    Z = Z_features[f]
    for img_id in range(len(Z)):
        voi_row = voi_df.loc[Z[img_id]]
        for aug_id in range(aug_factor):
            img = np.load(os.path.join(C.aug_dir, voi_row['cls'], "%s_%d.npy" % (Z[img_id], aug_id)))
            
            activ = model_dense.predict(np.expand_dims(img, 0))
            feature_dense[f] = np.concatenate([feature_dense[f], activ], axis=0)
        
            #activ = model_conv3.predict(np.expand_dims(img, 0))
            #feature_conv3_ch[f] = np.concatenate([feature_conv3_ch[f], activ.mean(axis=(1,2,3))], axis=0)
            #feature_conv3_sh[f] = np.concatenate([feature_conv3_sh[f], get_shells(activ, D)], axis=0)
        
            #activ = model_conv2.predict(np.expand_dims(img, 0))
            #feature_conv2_ch[f] = np.concatenate([feature_conv2_ch[f], activ.mean(axis=(1,2,3))], axis=0)

            #activ = model_conv1.predict(np.expand_dims(img, 0))
            #feature_conv1_ch[f] = np.concatenate([feature_conv1_ch[f], activ.mean(axis=(1,2,3))], axis=0)

# Output data

In [11]:
df = pd.DataFrame(columns=['true_cls', 'pred_cls'] + \
            [s for i in range(1,5) for s in ['feature_%d' % i,'strength_%d' % i]])

In [16]:
all_neurons = all_dense#np.concatenate([all_conv1_ch, all_conv2_ch, all_conv3_ch, all_dense], axis=1)
m = all_neurons.mean(axis=0)
all_cov = np.cov(all_neurons.T)

num_neurons = all_neurons.shape[-1]

lnZ = np.empty(num_features)
f_m = np.empty((num_features, num_neurons))
f_cov = np.empty((num_features, num_neurons, num_neurons))
for f_ix in range(num_features):
    #f_neurons = np.concatenate([feature_conv1_ch[all_features[f_ix]],
    #                            feature_conv2_ch[all_features[f_ix]],
    #                            feature_conv3_ch[all_features[f_ix]],
    #                            feature_dense[all_features[f_ix]]], axis=1)
    f_neurons = feature_dense[all_features[f_ix]]
    f_m[f_ix] = f_neurons.mean(0)
    f_cov[f_ix] = np.cov(f_neurons.T)

    lnZ_f = -scipy.stats.multivariate_normal.logpdf(f_neurons, m, all_cov, allow_singular=True)
    adj = np.amax(lnZ_f)
    lnZ[f_ix] = np.log(np.mean(np.exp(lnZ_f - adj))) + adj

In [29]:
for img_ix in range(len(z_test)):
    test_dense = np.empty([0,100])
    test_conv3_ch = np.empty([0,128])
    test_conv3_sh = np.empty([0,128*4])
    test_conv2_ch = np.empty([0,128])
    test_conv1_ch = np.empty([0,64*3])
    z = z_test[img_ix]
    voi_row = voi_df.loc[z]
    cls = voi_row['cls']
    row = [cls]

    x = np.expand_dims(x_test[img_ix], axis=0)
    preds = model.predict(x, verbose=False)[0]
    for pred_cls, _ in sorted(zip(C.classes_to_include, preds), key=lambda x:x[1], reverse=True)[:1]:
        row.append(pred_cls)

    p_f = np.empty(num_features)
    aug_factor = 100
    for aug_id in range(aug_factor):
        img = np.load(os.path.join(C.aug_dir, cls, "%s_%d.npy" % (z, aug_id)))

        activ = model_dense.predict(np.expand_dims(img, 0))
        test_dense = np.concatenate([test_dense, activ], axis=0)

        #activ = model_conv3.predict(np.expand_dims(img, 0))
        #test_conv3_ch = np.concatenate([test_conv3_ch, activ.mean(axis=(1,2,3))], axis=0)
        #test_conv3_sh = np.concatenate([test_conv3_sh, get_shells(activ, D)], axis=0)
        
        #activ = model_conv2.predict(np.expand_dims(img, 0))
        #test_conv2_ch = np.concatenate([test_conv2_ch, activ.mean(axis=(1,2,3))], axis=0)
        
        #activ = model_conv1.predict(np.expand_dims(img, 0))
        #test_conv1_ch = np.concatenate([test_conv1_ch, activ.mean(axis=(1,2,3))], axis=0)

    test_neurons = test_dense#np.concatenate([test_conv3_ch, test_dense], axis=1) 
    #m_test = test_neurons.mean(axis=0)
    #test_cov = np.cov(test_neurons.T)

    p_f = np.empty(num_features)
    for f_ix in range(num_features):  
        #indices = np.random.randint(0,test_neurons.shape[0], 1000)
        samp = test_neurons#[indices] #scipy.random.multivariate_normal(m_test, test_cov, size=10000)
        lnphf = scipy.stats.multivariate_normal.logpdf(samp, f_m[f_ix], f_cov[f_ix], allow_singular=True)
        lnph = scipy.stats.multivariate_normal.logpdf(samp, m, all_cov, allow_singular=True)

        adj = np.amax(lnphf - lnph)
        p_f[f_ix] = np.log(np.mean(np.exp(lnphf - lnph - adj))) + adj# + lnZ[f_ix]
        
    evidence = {all_features[f_ix]: p_f[f_ix] for f_ix in range(num_features)}
    
    f1='infiltrative growth'
    f2='nodular growth'
    if evidence[f1] < evidence[f2]:
        evidence[f1] -= 10
    else:
        evidence[f2] -= 10
        
    f3='central scar'
    evidence[f3] -= 50
    
    f4='isointense on venous phase'
    f5='washout'
    if evidence[f4] < evidence[f5]:
        evidence.pop(f4)
    else:
        evidence.pop(f5)
    #top3 = np.array(sorted(evidence.items(), key=lambda x:x[1], reverse=True)[:3])[:,0]
    #if f4 not in top3:
    #    evidence.pop(f4)
    #if f5 not in top3:
    #    evidence.pop(f5)

    for f,strength in sorted(evidence.items(), key=lambda x:x[1], reverse=True)[:4]:
        row += [f, strength]
        
    if np.mean([row[-7], row[-5], row[-3]]) / 3 > row[-1]:
        row[-2] = ''
        
    if np.mean([row[-7], row[-5]]) / 3 > row[-3]:
        row[-4] = ''
        
    df.loc[z] = row

In [31]:
df.to_excel(join(target_dir,'features_dense_final2.xlsx'))

## univariate

all_neurons = all_conv3_ch #np.concatenate([all_conv1_ch, all_conv2_ch, all_conv3_ch, all_dense], axis=1)
m = all_neurons.mean(0)
all_std = all_neurons.std(0)

num_neurons = all_neurons.shape[-1]

Z_uni = np.empty(num_features)
f_m = np.empty((num_features, num_neurons))
f_std = np.empty((num_features, num_neurons))
for f_ix in range(num_features):
    f_neurons = feature_conv3_ch[all_features[f_ix]]
    f_m[f_ix] = f_neurons.mean(0)
    f_std[f_ix] = f_neurons.std(0)

    Z_f = scipy.stats.norm.pdf(f_neurons, m, all_std)
    Z_uni[f_ix] = np.mean(1/Z_f)

for img_ix in range(len(z_test)):
    test_dense = np.empty([0,100])
    test_conv3_ch = np.empty([0,128])
    test_conv3_sh = np.empty([0,128*4])
    test_conv2_ch = np.empty([0,128])
    test_conv1_ch = np.empty([0,64*3])
    z = z_test[img_ix]
    voi_row = voi_df.loc[z]
    cls = voi_row['cls']
    row = [cls]

    x = np.expand_dims(x_test[img_ix], axis=0)
    preds = model.predict(x, verbose=False)[0]
    for pred_cls, _ in sorted(zip(C.classes_to_include, preds), key=lambda x:x[1], reverse=True)[:1]:
        row.append(pred_cls)

    p_f = np.empty(num_features)
    aug_factor = 100
    for aug_id in range(aug_factor):
        img = np.load(os.path.join(C.aug_dir, cls, "%s_%d.npy" % (z, aug_id)))

        #activ = model_dense.predict(np.expand_dims(img, 0))
        #test_dense = np.concatenate([test_dense, activ], axis=0)

        activ = model_conv3.predict(np.expand_dims(img, 0))
        test_conv3_ch = np.concatenate([test_conv3_ch, activ.mean(axis=(1,2,3))], axis=0)
        #test_conv3_sh = np.concatenate([test_conv3_sh, get_shells(activ, D)], axis=0)
        
        #activ = model_conv2.predict(np.expand_dims(img, 0))
        #test_conv2_ch = np.concatenate([test_conv2_ch, activ.mean(axis=(1,2,3))], axis=0)
        
        #activ = model_conv1.predict(np.expand_dims(img, 0))
        #test_conv1_ch = np.concatenate([test_conv1_ch, activ.mean(axis=(1,2,3))], axis=0)

    test_neurons = test_conv3_ch #np.concatenate([test_conv1_ch, test_conv2_ch, test_conv3_ch, test_dense], axis=1) 
    #m_test = test_neurons.mean(axis=0)
    #test_cov = np.cov(test_neurons.T)

    p_f = np.empty(num_features)
    for f_ix in range(num_features):        
        samp = test_neurons#scipy.random.multivariate_normal(m_test, test_cov, size=10000)
        phf = scipy.stats.norm.pdf(samp, f_m[f_ix], f_std[f_ix])
        ph = scipy.stats.norm.pdf(samp, m, all_std)

        p_f[f_ix] = np.mean(phf/ph)# * Z_uni[f_ix]
        
    evidence = {all_features[f_ix]: p_f[f_ix] for f_ix in range(num_features)}
    
    f1='infiltrative'
    f2='lobulated margins'
    if evidence[f1] < evidence[f2]:
        evidence.pop(f1)
    else:
        evidence.pop(f2)
        
    f1='homogeneous'
    f2='heterogeneous'
    if evidence[f1] < evidence[f2]:
        evidence.pop(f1)
    else:
        evidence.pop(f2)
        
    f3='central scar'
    f4='delayed isointensity'
    f5='arterial enhancement'
    top2 = np.array(sorted(evidence.items(), key=lambda x:x[1], reverse=True)[:2])[:,0]
    if f4 not in top2 and f5 not in top2:
        evidence.pop(f3)

    for f,strength in sorted(evidence.items(), key=lambda x:x[1], reverse=True)[:4]:
        row += [f, strength]
        
    df.loc[z] = row

df.to_csv(join(target_dir,'features_dense_uni1.csv'))

# Sample

m = all_neurons.mean(axis=0)
all_cov = np.cov(all_neurons.T)

test_dense = np.empty([0,100])
test_conv3 = np.empty([0,128*4])

p_f = np.empty(num_features)
img_id = 37
voi_row = voi_df.loc[z_test[img_id]]
cls = voi_row['cls']
aug_factor = 100
for aug_id in range(aug_factor):
    img = np.load(os.path.join(C.aug_dir, cls, "%s_%d.npy" % (z_test[img_id], aug_id)))
    
    activ = model_pre_act.predict(np.expand_dims(img, 0))
    test_dense = np.concatenate([test_dense, activ], axis=0)
    
    activ = model_conv3.predict(np.expand_dims(img, 0))
    test_conv3 = np.concatenate([test_conv3, get_shells(activ, D)], axis=0)

test_neurons = np.concatenate([test_conv3, test_dense], axis=1)
    
m_test = test_neurons.mean(axis=0)

c_ix = C.classes_to_include.index(cls)
test_cov = np.cov(test_neurons.T)

p_f = np.empty(num_features)
for f_ix in range(num_features):
    f_neurons = feature_neurons[all_features[f_ix]]
    f_m = f_neurons.mean(0)
    f_cov = np.cov(f_neurons.T)

    samp = scipy.random.multivariate_normal(m_test, test_cov, size=10000)
    lnphf = scipy.stats.multivariate_normal.logpdf(samp, f_m, f_cov, allow_singular=True)
    lnph = scipy.stats.multivariate_normal.logpdf(samp, m, all_cov, allow_singular=True)
    
    adj = np.max(lnphf - lnph)
    p_f[f_ix] = np.log(np.mean(np.exp(lnphf - lnph - adj))) + adj

print(cls)
for f,strength in sorted(enumerate(p_f), key=lambda x:x[1], reverse=True):
    #if strength<0.2:
    #    break
    print("%s %d" % (all_features[f], strength))

x_test_quick = x_test[img_id]#orig_data_dict[cls][0][np.where(orig_data_dict[cls][1] == indices_f[test_ix])]
hf.draw_slices(x_test_quick)

In [1]:
from os.path import *

import os
import pandas as pd
import glob
import shutil

In [2]:
xls = "F:\ReArranged-ITS.xlsx"

In [28]:
df =pd.read_excel(xls)

In [29]:
df["MRN-short"] = df["MRN-short"].astype(int)

In [40]:
for f in glob.glob(join("F:\\FileCopies","*")):
    #if len(basename(f)) > 8:
    g = basename(f)
    os.rename(f, f+"-"+str(df[df["MRN-short"] == int(g)]["ID"].values[0]))
    #print(df[df["MRN-short"] == int(g)]["ID"].values[0])

# Analysis

In [32]:
data_dir = r"C:\Users\Clinton\Documents\voi-classifier\data"
answer_key = join(data_dir, "ground_truth.xlsx")
answer_key = pd.read_excel(answer_key, index_col=0)

In [33]:
target_dir = "D:\\feature_analysis"
xls_path = join(target_dir, "features_dense_final2.xlsx")
df = pd.read_excel(xls_path, index_col=0)
#csv_path = join(target_dir, "features_dense_final1.csv")
#df = pd.read_csv(csv_path, index_col=0)

for f in sorted(all_features):
    num, prec_den, rec_den = 0,0,0
    for key, row in answer_key.iterrows():
        answer_features = row[['feature_1', 'feature_2', 'feature_3', 'feature_4']].dropna().values
        ignore_features = row[['ignore_1', 'ignore_2']].dropna().values
        pred_features = df.loc[key][['feature_1', 'feature_2', 'feature_3', 'feature_4']].values
        pred_str = df.loc[key][['strength_1', 'strength_2', 'strength_3', 'strength_4']].values
        if f in pred_features and f in answer_features:
            num += 1
        if f in pred_features and f not in ignore_features:
            prec_den += 1
        if f in answer_features:
            rec_den += 1
    print("%s,%d,%d,%d" % (f, num, prec_den, rec_den))

arterial phase enhancement,17,18,19
central scar,1,5,1
enhancing rim/capsule,5,8,15
heterogeneous texture,13,19,17
hyperintense mass on delayed phase,8,9,8
hypointense mass without enhancement,9,9,10
infiltrative growth,2,6,4
isointense on venous phase,8,15,9
nodular growth,5,14,6
nodular or discontinuous enhancement,8,11,10
progressive centripetal filling,9,14,9
progressive uniform enhancement,5,7,19
sharp margins,11,19,11
spherical hypointense mass,13,15,20
washout,8,15,9


In [178]:
num, prec_den, rec_den = 0,0,0
n=0
for key, row in answer_key.iterrows():
    answer_features = row[['feature_1', 'feature_2', 'feature_3', 'feature_4']].dropna().values
    for f in answer_features:
        if f not in all_features:
            print(f)
    pred_features = df.loc[key][['feature_1', 'feature_2', 'feature_3', 'feature_4']].values
    pred_str = df.loc[key][['strength_1', 'strength_2', 'strength_3', 'strength_4']].values
    if df.loc[key]['true_cls'] == df.loc[key]['pred_cls']:
        continue
    if np.mean(pred_str) - pred_str[-1] > 20:
        pred_features = pred_features[:-1]
    num += len([f for f in answer_features if f in pred_features])
    prec_den += len([f for f in pred_features if f not in ignore_features])
    rec_den += len(answer_features)
    n+=1
print(n, num, prec_den, rec_den)

60 48 211 193


In [ ]:
num, prec_den, rec_den = 0,0,0
n=0
for key, row in answer_key.iterrows():
    answer_features = row[['feature_1', 'feature_2', 'feature_3', 'feature_4']].dropna().values
    for f in answer_features:
        if f not in all_features:
            print(f)
    pred_features = df.loc[key][['feature_1', 'feature_2', 'feature_3', 'feature_4']].values
    pred_str = df.loc[key][['strength_1', 'strength_2', 'strength_3', 'strength_4']].values
    if np.mean(pred_str) - pred_str[-1] > 20:
        pred_features = pred_features[:-1]
    if pred_features[0] in answer_features:
        num += 1
    prec_den += len([f for f in pred_features if f not in ignore_features])
    rec_den += len(answer_features)
    n+=1
print(n, num, prec_den, rec_den)

In [35]:
corr = 0
for cls in sorted(C.classes_to_include):
    num, prec_den, rec_den = 0,0,0
    for key, row in answer_key.iterrows():
        if row['true_cls'] != cls:
            continue
        answer_features = row[['feature_1', 'feature_2', 'feature_3', 'feature_4']].dropna().values
        for f in answer_features:
            if f not in all_features:
                print(f)
        pred_features = df.loc[key][['feature_1', 'feature_2', 'feature_3', 'feature_4']].values
        pred_str = df.loc[key][['strength_1', 'strength_2', 'strength_3', 'strength_4']].values
        if np.mean(pred_str) - pred_str[-1] > 20:
            pred_features = pred_features[:-1]
        num += len([f for f in answer_features if f in pred_features])
        prec_den += len([f for f in pred_features if f not in ignore_features])
        rec_den += len(answer_features)
        #corr += len([f for f in answer_features if f in pred_features]) / len(pred_features)
    #corr/60
    print(cls, num, prec_den, rec_den)#, round(num/prec_den*100), round(num/rec_den*100))

cholangio 18 33 30
colorectal 11 35 27
cyst 24 30 27
fnh 17 36 20
hcc 23 35 32
hemangioma 28 26 31


In [137]:
133/193

0.689119170984456

answer_features = hf.flatten(answer_key[['feature_1', 'feature_2', 'feature_3', 'feature_4']].values)
unique, counts = np.unique(answer_features, return_counts=True)
base_freq = dict(zip(unique, counts))
base_freq.pop('nan')

features = hf.flatten(df[['feature_1', 'feature_2', 'feature_3', 'feature_4']].values)
unique, counts = np.unique(features, return_counts=True)
freq = dict(zip(unique, counts))
for f in base_freq.keys():
    if f in freq:
        print(f, freq[f])
    else:
        print(f, 0)

# Backup

In [170]:
x = np.linspace(-3,3, num=1000)

In [171]:
y = scipy.stats.norm.pdf(x, loc=m[0],scale=s[0])

In [195]:
F = feature_neurons[all_features[0]]
y2 = scipy.stats.norm.pdf(x, loc=F.mean(axis=0)[0],scale=F.std(axis=0)[0])

In [194]:
F.mean(axis=0)[0]

-0.5886752939191502

In [204]:
F = feature_neurons[all_features[2]]
y3 = scipy.stats.norm.pdf(x, loc=F.mean(axis=0)[0],scale=F.std(axis=0)[0])

In [209]:
F = feature_neurons[all_features[10]]
y4 = scipy.stats.norm.pdf(x, loc=F.mean(axis=0)[0],scale=F.std(axis=0)[0])

In [227]:
y_test = scipy.stats.norm.pdf(x, loc=m_test[0],scale=s_test[0])

In [ ]:
plt.figure(figsize=(3,2))
plt.plot(x,y,linewidth=2)
plt.plot(x,y2,linewidth=2)
plt.plot(x,y_test,linewidth=2)

In [ ]:
#plt.figure(figsize=(8,7))
plt.plot(x,y,linewidth=5, label='population')
plt.plot(x,y2,linewidth=2, label=all_features[0])
plt.plot(x,y3,linewidth=2, label=all_features[2])
plt.plot(x,y4,linewidth=2, label=all_features[10])
plt.axis([-3,3,0,1.2])
plt.legend()

#information provided by a feature about the possible values of a neuron, higher KL means that neuron should be weighted more
KL = np.zeros((num_features, num_neurons))
m = all_neurons.mean(axis=0)
s = all_neurons.std(axis=0)
for f_ix in range(num_features):
    F = feature_neurons[all_features[f_ix]]
    KL[f_ix, :] = kl_div(m,s, F.mean(axis=0),F.std(axis=0), one_sided="less")
    #KL[f_ix, :] = ttest_ind(F, all_pre_act, equal_var=False)

In [ ]:
df = pd.read_excel(os.path.join(C.base_dir, "data", "annotated_features.xlsx"), 5)

for accnum, row in df.iterrows():
    df = df.drop(accnum)
    accnum = prv.decode(accnum[:accnum.find('_')]) + accnum[accnum.find('_'):accnum.find(' ')]
    df.loc[accnum] = row

print('\n'.join([x for x in df.index]))

In [193]:
Z_test_features = ['E106097391_0', 'E104978772_1', '12900535_0', 'E100150242_0', 'E105490014_0', 'E103147618_0', 'E103510187_0', 'E104657225_0', 'E100551966_0', 'E101388602_0', 'E100215900_8', 'E100215900_7', 'E104045692_0', '13104521_0', 'E100383453_0', '12943286_0', '12271995_0', 'E102315724_0', 'E104949189_0', 'E100511083_1', 'E101579471_0', '13018986_1', '13203550_8', '13112385_0', '12712463_0', '12361082_0', '13028374_0', 'E103985934_1', 'E100529980_0', '12042703_3', '12961059_0', 'E105724706_2', 'E100592424_2', 'E103104254_0', 'E104546069_0', 'E101665217_1', '12090000_0', 'E100592424_1', '12961059_1', 'E105474285_0', '12502068_1', 'E100814791_0', 'E102613189_0', 'E105427046_0', 'E102881031_1', 'E102929168_0', 'E102310482_0', 'E102095465_0', 'E101811299_0', 'E104737273_0', '12890053_0', 'E100168661_1', '12637865_0', 'E100168661_2', '12239783_0', '12707781_0', '12706568_1', '12823036_0', '12404081_0', '12365693_1']

x_test = {cls: orig_data_dict[cls][0][np.where(np.isin(orig_data_dict[cls][1], Z_test_features))] for cls in C.classes_to_include}
Z_test = {cls: orig_data_dict[cls][1][np.where(np.isin(orig_data_dict[cls][1], Z_test_features))] for cls in C.classes_to_include}

filters_test = {}
features_test = {}
for cls in C.classes_to_include:
    filters_test[cls] = model_dense_outputs.predict(x_test[cls], verbose=False)

In [ ]:
for f_ix in range(num_features):

Requirements
- feature_vectors * unit_relevance should be maximized

- np.dot(feature_vectors[i], feature_vectors[j]) should be minimized, OR
- vec_distance(feature_vectors[i], feature_vectors[j]) should be maximized for all pairs i,j

- features "turn on / off" specific units; try to minimize the number of units impacted by a given feature

- p(z|x) > .75 for all x manually annotated by z

===
- show % of evidence explained (fraction of sum of contributing units that are captured by features that turn those units on)

header = ['filter_num']
for cls in C.classes_to_include:
    header += [f+"_"+cls for f in features_by_cls[cls]]

with open('E:\\feature_filters.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for f_num in range(100):
        writer.writerow([f_num] + [feature_filters[f][f_num] for cls in features for f in features_by_cls[cls]])

cls = "colorectal"
x_test_quick = orig_data_dict[cls][0][np.where(orig_data_dict[cls][1] == "E105724706_2.npy")]
x_test_quick = orig_data_dict["fnh"][0][np.where(orig_data_dict["fnh"][1] == "E104189184_0.npy")]
filters_quick = model_dense_outputs.predict(x_test_quick, verbose=False)

In [ ]:
img_num = 0
evidence = {}

for f in all_features:
    evidence[f + "/" + str(cls_features[f])] = cnna.get_evidence_strength(feature_filters[f], filters_quick[0])#filters_test[true_cls][img_num])
    #max_strength = max(max_strength, evidence[f + "/" + str(cls_features[f])])

#for f in evidence:
#    evidence[f] /= max_strength
print("Detected features:")
for f,strength in sorted(evidence.items(), key=lambda x:x[1], reverse=True)[:5]:
    #if strength > 1:
    print("- " + f, "- %d%%" % (strength*100))

hf.plot_section_auto(x_test_quick[0])#[true_cls][img_num])

In [104]:
with open('E:\\filters_pred6.csv', 'w', newline='') as csvfile:
    header = ['img_fn', 'agreement1', 'agreement2', 'true_cls', 'pred_cls1', 'pred_cls2'] + \
            [s for i in range(len(all_features)) for s in ['feature_%d' % i,'strength_%d' % i]]
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for z_num in range(len(Z_test_features)):
        writer.writerow([Z_test_features[z_num]] + [output[Z_test_features[z_num]][0] in output[Z_test_features[z_num]][3], \
                        output[Z_test_features[z_num]][0] in output[Z_test_features[z_num]][5]] + output[Z_test_features[z_num]])

In [60]:
df = pd.read_excel("E:\\filters_pred3.xlsx")

In [75]:
agree = 0
for _,row in df.iterrows():
    if row["pred_cls1"] in row["feature_1"]:
        agree += 1
print(agree/60)

52


# NMF

In [ ]:
nmf = sklearn.decomposition.NMF(10)

In [ ]:
nmf.fit_transform()

## Relevance calculations

In [107]:
W = model.layers[-3].get_weights()[0]
bias = model.layers[-3].get_weights()[1]

#np.dot(filter_results[0], W) * eff_mult + eff_bias

gamma, beta, mu, var = model.layers[-2].get_weights()

eff_bias = (np.zeros(6) + bias - mu) / var**.5 * gamma + beta
eff_mult = (np.ones(6) + bias - mu) / var**.5 * gamma + beta - eff_bias

W_eff = W * eff_mult# + eff_bias

#(np.dot(filter_results[0], W) + bias - mu) / var**.5 * gamma + beta

In [117]:
unit_relevance = np.empty(num_units)
discr_power = np.empty((num_units, 6))
for u_ix in range(num_units):
    unit_relevance[u_ix] = np.amax(W_eff[u_ix]) - np.amin(W_eff[u_ix])
    for c_ix in range(6):
        if W_eff[u_ix, c_ix] == np.amax(W_eff[u_ix]):
            discr_power[u_ix, c_ix] = W_eff[u_ix, c_ix] - sorted(W_eff[u_ix],reverse=True)[1]
        else:
            discr_power[u_ix, c_ix] = W_eff[u_ix, c_ix] - np.amax(W_eff[u_ix])

In [209]:
filter_results = filter_results*unit_relevance

In [217]:
avg_length = np.mean(np.apply_along_axis(get_length, 1, filter_results*unit_relevance))

In [219]:
np.sum(squash(filter_results[2] * unit_relevance * 2/avg_length)**2)

0.6239091267665856